In [1]:
import cv2
import numpy as np 
import pandas as pd 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input

In [26]:
labels = pd.read_csv("labels.csv")
train_file = 'train/'
test_file = 'test/'

In [27]:
print("Total number of unique Dog Breeds :",len(labels.breed.unique()))

Total number of unique Dog Breeds : 120


In [28]:
num_breeds = 60
im_size = 224
batch_size = 64
encoder = LabelEncoder()

In [29]:
#creating new list for 60 breeds
breed_dict = list(labels['breed'].value_counts().keys()) 
new_list = sorted(breed_dict,reverse=True)[:num_breeds*2+1:2]
labels = labels.query('breed in @new_list')

In [30]:
labels['img_file'] = labels['id'].apply(lambda x: x + ".jpg")


In [32]:
#Encoding and Scaleing imagess
train_x = np.zeros((len(labels), im_size, im_size, 3), dtype='float32')
 

for i, img_id in enumerate(labels['img_file']):
  
  img = cv2.resize(cv2.imread(train_file+img_id,cv2.IMREAD_COLOR),((im_size,im_size)))
 
  img_array = preprocess_input(np.expand_dims(np.array(img[...,::-1].astype(np.float32)).copy(), axis=0))
  #update the train_x variable with new element
  train_x[i] = img_array

In [33]:
train_y = encoder.fit_transform(labels["breed"].values)

In [34]:
#training and testing data
x_train, x_test, y_train, y_test = train_test_split(train_x,train_y,test_size=0.2,random_state=42)

In [36]:
train_datagen = ImageDataGenerator(
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.25,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow(
    x_train,
    y_train,
    batch_size=batch_size
)

test_generator = test_datagen.flow(
    x_test,
    y_test,
    batch_size=batch_size
)

In [38]:
resnet = ResNet50V2(input_shape = [im_size,im_size,3], weights='imagenet', include_top=False)
#freeze all trainable layers and train only top layers 
for layer in resnet.layers:
    layer.trainable = False

x = resnet.output
x = BatchNormalization()(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
#add fully connected layer
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)

In [39]:
predictions = Dense(num_breeds, activation='relu')(x)
 
#create model class with inputs and outputs
model = Model(inputs=resnet.input, outputs=predictions)
#model.summary()

In [41]:
epochs = 20
learning_rate = 1e-3
 
#using RMSprop optimizer to compile or build the model
optimizer = RMSprop(learning_rate=learning_rate,rho=0.9)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"])
 
#fit the training generator data and train the model
hist = model.fit(train_generator,
                 steps_per_epoch= x_train.shape[0] // batch_size,
                 epochs= epochs,
                 validation_data= test_generator,
                 validation_steps= x_test.shape[0] // batch_size)
 
#Save the model for prediction
model.save("model")

Epoch 1/2
64/64 [==============================] - 461s 7s/step - loss: 4.4284 - accuracy: 0.0231 - val_loss: 4.1143 - val_accuracy: 0.0283
Epoch 2/2
64/64 [==============================] - 349s 5s/step - loss: 4.1585 - accuracy: 0.0233 - val_loss: 4.1128 - val_accuracy: 0.0264


INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets


In [42]:
import os

model = load_model("model")
 
#get the image of the dog for prediction
pred_img_path = 'afghan_hound.jpeg'

# check that the file exists
if not os.path.exists(pred_img_path):
    print(f"Error: File '{pred_img_path}' not found")
    exit(1)

#read the image file and convert into numeric format
#resize all images to one dimension i.e. 224x224
pred_img = cv2.imread(pred_img_path,cv2.IMREAD_COLOR)

# check that the image was read successfully
if pred_img is None:
    print(f"Error: Failed to read image '{pred_img_path}'")
    exit(1)

# check that the image has a valid size
if pred_img.shape[0] < im_size or pred_img.shape[1] < im_size:
    print(f"Error: Image '{pred_img_path}' is too small")
    exit(1)

pred_img_array = cv2.resize(pred_img,((im_size,im_size)))

#scale array into the range of -1 to 1.
#expand the dimension on the axis 0 and normalize the array values
pred_img_array = preprocess_input(np.expand_dims(np.array(pred_img_array[...,::-1].astype(np.float32)).copy(), axis=0))
 
#feed the model with the image array for prediction
pred_val = model.predict(np.array(pred_img_array,dtype="float32"))
 
#display the image of dog
cv2.imshow("TechVidvan", cv2.resize(pred_img,(im_size,im_size))) 
 
#display the predicted breed of dog
pred_breed = sorted(new_list)[np.argmax(pred_val)]
print("Predicted Breed for this Dog is :",pred_breed)

1/1 [==============================] - 1s 1s/step
Predicted Breed for this Dog is : afghan_hound
